In [ ]:
import cv2
import numpy as np
import os


In [ ]:
# --- Configuration ---
main_image_path = 'barbara.jpg'
patch_image_path = 'patch.png'
output_image_path = 'barbara_with_patch_box.jpg'

In [ ]:
def calculate_euclidean_distance(patch1, patch2):
    """Calculates the Euclidean distance between two image patches."""
    # Ensure patches have the same shape
    if patch1.shape != patch2.shape:
        raise ValueError("Patches must have the same dimensions and channels.")
    
    # Flatten the patches and calculate distance
    # Alternatively, calculate squared distance directly on arrays for efficiency
    # dist = np.sqrt(np.sum((patch1.astype(float) - patch2.astype(float))**2))
    
    # Using squared Euclidean distance for comparison (avoids costly sqrt)
    # Convert to float to prevent overflow issues with uint8 subtraction
    dist_sq = np.sum((patch1.astype(np.float64) - patch2.astype(np.float64))**2)
    return dist_sq

In [ ]:
print(f"Loading main image: {main_image_path}")
if not os.path.exists(main_image_path):
    print(f"Error: Main image file not found at {main_image_path}")
    exit()
main_img = cv2.imread(main_image_path)
if main_img is None:
    print(f"Error: Could not load main image {main_image_path}")
    exit()

print(f"Loading patch image: {patch_image_path}")
if not os.path.exists(patch_image_path):
    print(f"Error: Patch image file not found at {patch_image_path}")
    exit()
patch_img = cv2.imread(patch_image_path)
if patch_img is None:
    print(f"Error: Could not load patch image {patch_image_path}")
    exit()

In [ ]:
main_h, main_w = main_img.shape[:2]
patch_h, patch_w = patch_img.shape[:2]

print(f"Main image dimensions (HxW): {main_h} x {main_w}")
print(f"Patch image dimensions (HxW): {patch_h} x {patch_w}")

# Check if patch is larger than the main image
if patch_h > main_h or patch_w > main_w:
    print("Error: Patch image is larger than the main image.")
    exit()

In [ ]:
min_distance = np.inf
best_match_coords = (0, 0) # (y, x) of top-left corner

print("Searching for the best match using Euclidean distance...")
# Iterate through all possible top-left corners (y, x) for the patch
for y in range(main_h - patch_h + 1):
    for x in range(main_w - patch_w + 1):
        # Extract the current window/candidate patch from the main image
        candidate_patch = main_img[y : y + patch_h, x : x + patch_w]
        
        # Calculate the distance (using squared Euclidean for efficiency)
        distance = calculate_euclidean_distance(patch_img, candidate_patch)
        
        # Update if this is a better match
        if distance < min_distance:
            min_distance = distance
            best_match_coords = (y, x) # Store NumPy-style coordinates (row, col)

In [ ]:
best_y, best_x = best_match_coords
top_left = (best_x, best_y)         # OpenCV uses (x, y) for drawing
bottom_right = (best_x + patch_w, best_y + patch_h)

print(f"Best match found!")
print(f" -> Minimum Squared Euclidean Distance: {min_distance:.2f}")
print(f" -> Top-left corner (x, y): {top_left}")
print(f" -> Bottom-right corner (x, y): {bottom_right}")

In [ ]:
output_img = main_img.copy()
box_color = (0, 0, 255) # BGR format for Red
box_thickness = 2
cv2.rectangle(output_img, top_left, bottom_right, box_color, box_thickness)


In [ ]:
print(f"Saving result with bounding box to: {output_image_path}")
cv2.imwrite(output_image_path, output_img)

print("Displaying result. Press any key to close.")
cv2.imshow('Main Image', main_img)
cv2.imshow('Patch', patch_img)
cv2.imshow('Found Patch Location', output_img)

# Wait for a key press and then close windows
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Done.")